In [ ]:
import numpy as np
import pandas as pd

from astropy.time import Time
from astropy.coordinates import Angle
from astropy import units as u

import matplotlib.pylab as plt
#%matplotlib widget

In [ ]:
#plt.figure()
#plt.scatter([0,1], [0,1])

In [ ]:
#import rubin_sim

In [ ]:
#rubin_sim

In [ ]:
from rubin_sim.scheduler.model_observatory import ModelObservatory
from rubin_sim.scheduler.surveys import generate_dd_surveys, DeepDrillingSurvey, FieldSurvey
from rubin_sim.scheduler.schedulers import CoreScheduler
from rubin_sim.scheduler.utils import empty_observation
from rubin_sim.scheduler.detailers import DitherDetailer
from rubin_sim.scheduler.basis_functions import M5DiffBasisFunction
from rubin_sim.utils import ddf_locations
from rubin_sim.scheduler import sim_runner

In [ ]:
# This does not work yet
# from lsst.daf.butler import Butler

# Single Field

In [ ]:
"""
class SingleFieldSurvey(BaseSurvey):
    
    def __init__(
        self,
        basis_functions,
        RA,
        dec,
        sequence="rgizy",
        nvis=[20, 10, 20, 26, 20],
        exptime=30.0,
        u_exptime=30.0,
        nexp=2,
        ignore_obs=None,
        survey_name="SingleFieldSurvey",
        reward_value=None,
        readtime=2.0,
        filter_change_time=120.0,
        nside=None,
        flush_pad=30.0,
        seed=42,
        detailers=None,
    ):
"""

In [ ]:
# We need the start date of the survey, so let's load up our model observatory and get that from the conditions
nside = 32  # Specify the HEALpix resolution
mo = ModelObservatory(nside=nside)

In [ ]:
conditions = mo.return_conditions()

In [ ]:
conditions

In [ ]:
t = Time(conditions.mjd, format='mjd')
t.to_value('iso')

In [ ]:
# Will need to update the mounted filters to be ComCam specific
conditions.mounted_filters

In [ ]:
np.degrees(conditions.sun_alt)

In [ ]:
Angle(conditions.lmst, u.hr).deg

In [ ]:
help(conditions)

In [ ]:
surveys = generate_dd_surveys()

In [ ]:
surveys

In [ ]:
np.degrees(surveys[3].ra)

In [ ]:
dir(surveys[0].basis_functions[5])

In [ ]:
surveys[3].add_observation(empty_observation())

In [ ]:
surveys[3].generate_observations(conditions)

In [ ]:
from rubin_sim.scheduler import features

In [ ]:
features.NObsSurvey(note='hello').feature

In [ ]:
from rubin_sim.scheduler.basis_functions import LookAheadDdfBasisFunction

In [ ]:
LookAheadDdfBasisFunction

In [ ]:
surveys[3].basis_functions[5].survey_name

In [ ]:
locations = ddf_locations()

survey_name = "DD:COSMOS"
RA = locations["COSMOS"][0]
dec = locations["COSMOS"][1]
ha_limits = ([0.0, 2.5], [21.5, 24.0])
print(RA, dec)

In [ ]:
# Basis functions
bfs = []

In [ ]:
# Add in detailer
#detailers = None
detailers = [DitherDetailer(max_dither=0.7, per_night=False)]

In [ ]:
#nvis_master = [8, 20, 10, 20, 26, 20]
nvis_master = [5, 5, 5, 5, 5, 5]
exptime = 30
u_exptime = 30
reward_value = 100
nexp = 2

surveys = []

surveys.append(
        DeepDrillingSurvey(
            bfs,
            RA,
            dec,
            sequence="urgizy",
            nvis=nvis_master,
            exptime=exptime,
            u_exptime=u_exptime,
            survey_name=survey_name,
            reward_value=reward_value,
            nside=nside,
            nexp=nexp,
            detailers=detailers,
        )
    )

In [ ]:
surveys[0]

In [ ]:
scheduler = CoreScheduler(surveys, nside=nside) # camera="comcam"

In [ ]:
# kludge to avoid divide by zero error
# scheduler.add_observation(empty_observation())

In [ ]:
surveys[0].generate_observations(conditions)

In [ ]:
# Below if the demo syntax
#mo, scheduler, observations = sim_runner(mo, scheduler, survey_length=0.15, verbose=True)

In [ ]:
# Note the deep copy used here. The function sim_runner modifies the observatory
import copy
new_mo, new_scheduler, observations = sim_runner(copy.deepcopy(mo), scheduler, survey_length=0.15, verbose=True)

In [ ]:
mo.mjd

In [ ]:
pd.DataFrame(np.hstack(observations))

In [ ]:
plt.figure(dpi=200)
plt.plot(observations['mjd'], np.degrees(observations['alt']), 'ko', markersize=1)
plt.xlabel('MJD')
plt.ylabel('Altitude (degrees)')

In [ ]:
f2c = {'u': 'purple', 'g': 'blue', 'r': 'green',
       'i': 'cyan', 'z': 'orange', 'y': 'red'}

plt.figure(dpi=200)
for filtername in f2c:
    in_filt = np.where(observations['filter'] == filtername)[0]
    if in_filt.size > 0:
        plt.plot(observations['mjd'][in_filt], np.degrees(observations['alt'][in_filt]), 
                 'o', markersize=1, color=f2c[filtername], label=filtername)
plt.legend()
plt.xlabel('MJD')
plt.ylabel('Altitude (degrees)')

In [ ]:
plt.figure(dpi=200)
for filtername in f2c:
    in_filt = np.where(observations['filter'] == filtername)[0]
    if in_filt.size > 0:
        plt.plot(np.degrees(observations['RA'][in_filt]), np.degrees(observations['dec'][in_filt]), 
                 'o', markersize=1, color=f2c[filtername], label=filtername)
plt.legend()
plt.xlabel('RA (degress)')
plt.ylabel('dec (degrees)')

In [ ]:
# Load Gaia refact
# https://github.com/LSSTScienceCollaborations/StackClub/blob/master/Validation/DC2_refcat_loader_demo.ipynb
# https://github.com/lsst/analysis_tools/blob/main/python/lsst/analysis/tools/tasks/catalogMatch.py
# /sdf/group/rubin/datasets/refcats/htm/v1/
# Number of stars in focal plane coordinates
# Number of observations in sky coordinates

# Stellar Density

In [ ]:
# Data directory
import os
print(os.getenv("RUBIN_SIM_DATA_DIR"))

In [ ]:
m = np.load('/home/b/bechtol/rubin_sim_data/maps/StarMaps/starDensity_g_nside_64.npz')

In [ ]:
m['starDensity'].shape

In [ ]:
import healpy as hp

In [ ]:
hp.nside2npix(64)

In [ ]:
m['bins']

In [ ]:
hp.mollview(m['starDensity'][:,20])

# Multiple Fields

In [ ]:
# We need the start date of the survey, so let's load up our model observatory and get that from the conditions
nside = 32  # Specify the HEALpix resolution
mo = ModelObservatory(nside=nside)
conditions = mo.return_conditions()

In [ ]:
ra_array = np.arange(0., 360., 30.)
dec_array = np.tile(-20., len(ra_array))
print(ra_array)
print(dec_array)

In [ ]:
fields = {}
for ra, dec in zip(ra_array, dec_array):
    name = f"{int(ra)}{int(dec)}"
    fields[name] = {"RA": ra, "dec": dec}
print(fields)

In [ ]:
# Basis functions
#bfs = []
bfs = [M5DiffBasisFunction(filtername='r', nside=nside)]

In [ ]:
# Add in detailer
#detailers = None
detailers = [DitherDetailer(max_dither=0.7, per_night=False)]

In [ ]:
#nvis_master = [8, 20, 10, 20, 26, 20]
nvis_master = [5, 5, 5, 5, 5, 5]
exptime = 30
u_exptime = 30
reward_value = 100
nexp = 2

surveys = []

for survey_name in fields.keys():
    print(survey_name)
    #reward_value = 100 if fields[survey_name]["RA"] == 150. else 0
    reward_value = 100.
    #reward_value = np.random.random()
    #reward_value = None
    surveys.append(
        #DeepDrillingSurvey(
        FieldSurvey(
            bfs,
            fields[survey_name]["RA"],
            fields[survey_name]["dec"],
            sequence="urgizy",
            nvis=nvis_master,
            exptime=exptime,
            u_exptime=u_exptime,
            survey_name=survey_name,
            reward_value=reward_value,
            nside=nside,
            nexp=nexp,
            detailers=detailers,
        )
    )

In [ ]:
#surveys

In [ ]:
#surveys[5].generate_observations(conditions)

In [ ]:
#surveys = [surveys[5]]

In [ ]:
#surveys

In [ ]:
scheduler = CoreScheduler(surveys, nside=nside) # camera="comcam"

In [ ]:
#for survey in scheduler.survey_lists[0]:
#    #print(survey.generate_observations(conditions))
#    print(survey.calc_reward_function(conditions))

In [ ]:
#scheduler.survey_lists[0][0].generate_observations(conditions)

In [ ]:
# Note the deep copy used here. The function sim_runner modifies the observatory
import copy
new_mo, new_scheduler, observations = sim_runner(copy.deepcopy(mo), scheduler, survey_length=1., verbose=True)

In [ ]:
pd.DataFrame(np.hstack(observations))

In [ ]:
f2c = {'u': 'purple', 'g': 'blue', 'r': 'green',
       'i': 'cyan', 'z': 'orange', 'y': 'red'}

plt.figure(dpi=200)
for filtername in f2c:
    in_filt = np.where(observations['filter'] == filtername)[0]
    if in_filt.size > 0:
        plt.plot(observations['mjd'][in_filt], np.degrees(observations['alt'][in_filt]), 
                 'o', markersize=1, color=f2c[filtername], label=filtername)
plt.legend()
plt.xlabel('MJD')
plt.ylabel('Altitude (degrees)')

In [ ]:
plt.figure(dpi=200)
for filtername in f2c:
    in_filt = np.where(observations['filter'] == filtername)[0]
    if in_filt.size > 0:
        plt.plot(np.degrees(observations['RA'][in_filt]), np.degrees(observations['dec'][in_filt]), 
                 'o', markersize=1, color=f2c[filtername], label=filtername)
plt.legend()
plt.xlabel('RA (degress)')
plt.ylabel('dec (degrees)')

In [ ]:
plt.figure(dpi=200)
plt.scatter(np.degrees(observations['RA']), np.degrees(observations['dec']), c=observations['mjd'])
plt.colorbar(label='MJD')
plt.xlabel('RA (degress)')
plt.ylabel('dec (degrees)')